##### Carregando dataset

In [ ]:
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

df = pd.read_excel('data/stanfordMOOCForumPostsSet.xlsx')
df.head(3)

##### Carregando Transformer

In [ ]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
# translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt", device=-1) # device=-1 para CPU

##### Removendo null

In [ ]:
print(df['Text'].isnull().sum()) # Verifica valores nulos
# print(df['Text'].apply(len).describe()) # Verifica estatísticas de comprimento

In [ ]:
null_indices = df[df['Text'].isnull()].index.tolist()
null_indices

In [ ]:
df['Text'][18312] = ''
df['Text'][18312]

##### Removendo int

In [ ]:
df[df['Text'].apply(lambda x: type(x) == int)].index.tolist()

In [ ]:
df['Text'][11157] = ''
df['Text'][19732] = ''
df['Text'][23525] = ''
df['Text'][24285] = ''

##### Removendo float

In [ ]:
df[df["Text"].apply(lambda x: type(x) == float)].index.tolist()

In [28]:
df["Text"][27323] = ""

##### Removendo caracteres não Ascii

In [ ]:
def remove_non_ascii(text):
    return "".join(char for char in text if char.isascii())

df["Text"] = df["Text"].apply(lambda x: remove_non_ascii(x))

##### Traduções

In [ ]:
df["Text"][2]

In [ ]:
# traduzir um documento
translator(df["Text"][2], max_length=2048)[0]["translation_text"]

In [44]:
# teste de 10 traduções
df["Text"][0:10].apply(lambda x: translator(x, max_length=2048)[0]["translation_text"])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0    Interessante! Quantas vezes dizemos essas cois...
1    O que é "Álgebra como um Jogo de Matemática" o...
2    Eu gosto da ideia do diretor dos meus filhos q...
3    A partir de suas respostas, parece que os alun...
4    Os meninos adoraram a matemática, porque há li...
5    A ideia generalizada de que ser bom em matemát...
6    O módulo de revisão por pares ainda não está t...
7    Gostaria de começar o ano de folga compartilha...
8    Pedir aos alunos que ilustrem como os erros po...
9    Eu acho que isso cai no professor. Alguém me d...
Name: Text, dtype: object

In [45]:
# melhor uso da GPU
translated_posts = translator(
    df["Text"][0:10]
    .apply(lambda x: x if type(x) in (str, list) else "no text")
    .to_list(),
    max_length=2048,
)
translated_posts = [post["translation_text"] for post in translated_posts]
translated_posts

['Interessante! Quantas vezes dizemos essas coisas aos outros sem realmente entender o que estamos dizendo. Deve ter sido uma experiência poderosa! Excelente!',
 'O que é "Álgebra como um Jogo de Matemática" ou você está apenas dizendo que cria jogos que incorporam álgebra?',
 'Eu gosto da ideia do diretor dos meus filhos que diz “Smart não significa fácil, inteligente significa trabalhar duro” e incorpora a ideia de cometer erros para trabalhar duro.',
 'A partir de suas respostas, parece que os alunos realmente gostaram do poder que sentiam ser "livres" para resolver a matemática da maneira que quisessem. Eles usaram linguagem acadêmica como números em decomposição e números amigáveis e foram capazes de explicar o significado. "',
 'Os meninos adoraram a matemática, porque há liberdade, você pode fazer qualquer coisa." Que ótima maneira de ver matemática e números! Não acho que os alunos percebam que a matemática pode ser ensinada de maneira diferente. Eles só pensaram que é isso que

In [ ]:
# traduzir dataset completo
translated_posts = translator(
    df["Text"]
    .apply(lambda x: x if type(x) in (str, list) else "no text")
    .to_list(),
    max_length=2048,
)
translated_posts = [post["translation_text"] for post in translated_posts]

# salvar
df.insert(1, column="Text_PT", value=translated_posts)
df.to_excel('data/stanfordMOOCForumPostsSet_PT.xlsx', index=False)
df.head(3)

In [ ]:
# traduzir em lotes
batch_size = 100  # Ajuste este valor conforme necessário
translated_posts = []

for i in range(0, len(df), batch_size):
    batch = df["Text"][i : i + batch_size]
    translated_batch = translator(
        batch.apply(lambda x: x if type(x) in (str, list) else "no text").to_list(),
        max_length=2048,
    )
    translated_batch = [post["translation_text"] for post in translated_batch]
    translated_posts.extend(translated_batch)

df.insert(1, column="Text_PT", value=translated_posts)
df.to_excel('data/stanfordMOOCForumPostsSet_PT.xlsx', index=False)
df.head(3)